In [1]:
!pip install sentence-transformers pandas pinecone-client

In [2]:
!pip install yfinance

In [3]:
!pip install tf-keras

# Pinecone and Dataset

In [1]:
import os
import pandas as pd
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone
import logging
import yfinance as yf

C:\Users\Pham Ty\AppData\Roaming\Python\Python312\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
# Initialize logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [3]:
# Initialize Pinecone
pc = Pinecone(api_key="09bb980b-6cef-48c3-9aa5-63f3cbc9885e")
index = pc.Index("financial-data-index")

INFO:pinecone_plugin_interface.logging:Discovering subpackages in _NamespacePath(['C:\\Users\\Pham Ty\\AppData\\Roaming\\Python\\Python312\\site-packages\\pinecone_plugins'])
INFO:pinecone_plugin_interface.logging:Looking for plugins in pinecone_plugins.inference
INFO:pinecone_plugin_interface.logging:Installing plugin inference into Pinecone


In [4]:
# Initialize SentenceTransformer model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
logger.info("SentenceTransformer 'all-MiniLM-L6-v2' loaded successfully.")

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cuda
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
C:\Users\Pham Ty\AppData\Roaming\Python\Python312\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
INFO:__main__:SentenceTransformer 'all-MiniLM-L6-v2' loaded successfully.


In [5]:
# Folder containing the CSV files
folder_path = r'C:\Users\Pham Ty\Desktop\Thesis-Predict\Dataset-2'

# Folder to save the processed CSV files
output_folder = r'C:\Users\Pham Ty\Desktop\Thesis-Predict\Dataset_processed_2'

In [6]:
# Định nghĩa danh sách các mã chứng khoán
symbols = ['NVDA', 'INTC', 'PLTR', 'TSLA', 'AAPL', 'BBD', 'T', 'SOFI',
    'WBD', 'SNAP', 'NIO', 'BTG', 'F', 'AAL', 'NOK', 'BAC',
    'CCL', 'ORCL', 'AMD', 'PFE', 'KGC', 'MARA', 'SLB', 'NU',
    'MPW', 'MU', 'LCID', 'NCLH', 'RIG', 'AMZN', 'ABEV', 'U',
    'LUMN', 'AGNC', 'VZ', 'WBA', 'WFC', 'RIVN', 'UPST', 'GRAB',
    'CSCO', 'VALE', 'AVGO', 'PBR', 'GOOGL', 'SMMT', 'GOLD',
    'CMG', 'BCS', 'UAA']

In [7]:
def fetch_and_save_stock_data(symbol, start_date='2019-01-01', end_date='2024-01-01'):
    """
    Fetch stock data from Yahoo Finance, save to CSV, and upsert data into Pinecone.
    """
    try:
        # Fetch data from Yahoo Finance
        stock_data = yf.download(symbol, start=start_date, end=end_date)

        # Save data to CSV
        csv_file_path = os.path.join(folder_path, f'{symbol}.csv')
        stock_data.to_csv(csv_file_path)
        logger.info(f"Data for {symbol} saved to {csv_file_path}")

        # Load CSV data
        df = pd.read_csv(csv_file_path)

        # Select necessary columns and drop NaN values
        required_columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
        if not all(col in df.columns for col in required_columns):
            logger.error(f"{csv_file_path} is missing one or more required columns: {required_columns}")
            return

        df = df[required_columns].dropna()

        # Convert each row into a single string of concatenated data
        text_data = df.apply(
            lambda row: f"Date: {row['Date']}, Open: {row['Open']}, High: {row['High']}, Low: {row['Low']}, Close: {row['Close']}, Adj Close: {row['Adj Close']}, Volume: {row['Volume']}",
            axis=1
        )

        # Generate embeddings for the stock data
        embeddings = embedding_model.encode(text_data.tolist(), convert_to_numpy=True)

        # Prepare data for Pinecone upsert
        vectors = [(f"{symbol}_{i}", embeddings[i].tolist(), {'symbol': symbol}) for i in range(len(embeddings))]

        # Upsert data into Pinecone index
        try:
            response = index.upsert(vectors)
            logger.info(f"Successfully upserted {len(vectors)} vectors for {symbol}.")
            logger.info(f"Pinecone response: {response}")
        except Exception as e:
            logger.error(f"Error upserting data into Pinecone for {symbol}: {e}")

        # Save the processed DataFrame to a new CSV file in the output folder
        processed_file_path = os.path.join(output_folder, f'processed_{symbol}.csv')
        df.to_csv(processed_file_path, index=False)
        logger.info(f"Processed data saved to {processed_file_path}")

    except Exception as e:
        logger.error(f"Error processing data for {symbol}: {e}")

In [11]:
# Loop through all symbols and process each
for symbol in symbols:
    fetch_and_save_stock_data(symbol)

[*********************100%***********************]  1 of 1 completed
INFO:__main__:Data for NVDA saved to C:\Users\phamt\Downloads\Thesis_Project\Dataset\NVDA.csv


Batches:   0%|          | 0/40 [00:00<?, ?it/s]

ERROR:__main__:Error upserting data into Pinecone for NVDA: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Sat, 28 Sep 2024 14:07:37 GMT', 'Content-Type': 'application/json', 'Content-Length': '113', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '4184', 'x-pinecone-request-id': '8090517448422635836', 'x-envoy-upstream-service-time': '16', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"Number of provided vectors: 1258 exceeds the maximum amount per request: 1000","details":[]}

INFO:__main__:Processed data saved to C:\Users\phamt\Downloads\Thesis_Project\Processed_Dataset\processed_NVDA.csv
[*********************100%***********************]  1 of 1 completed
INFO:__main__:Data for INTC saved to C:\Users\phamt\Downloads\Thesis_Project\Dataset\INTC.csv


Batches:   0%|          | 0/40 [00:00<?, ?it/s]

ERROR:__main__:Error upserting data into Pinecone for INTC: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Sat, 28 Sep 2024 14:08:35 GMT', 'Content-Type': 'application/json', 'Content-Length': '113', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '2971', 'x-pinecone-request-id': '835272394218929297', 'x-envoy-upstream-service-time': '16', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"Number of provided vectors: 1258 exceeds the maximum amount per request: 1000","details":[]}

INFO:__main__:Processed data saved to C:\Users\phamt\Downloads\Thesis_Project\Processed_Dataset\processed_INTC.csv
[*********************100%***********************]  1 of 1 completed
INFO:__main__:Data for PLTR saved to C:\Users\phamt\Downloads\Thesis_Project\Dataset\PLTR.csv


...

ERROR:__main__:Error upserting data into Pinecone for CMG: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Sat, 28 Sep 2024 14:37:40 GMT', 'Content-Type': 'application/json', 'Content-Length': '113', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '1057', 'x-pinecone-request-id': '4907492978887113189', 'x-envoy-upstream-service-time': '48', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"Number of provided vectors: 1258 exceeds the maximum amount per request: 1000","details":[]}

INFO:__main__:Processed data saved to C:\Users\phamt\Downloads\Thesis_Project\Processed_Dataset\processed_CMG.csv
[*********************100%***********************]  1 of 1 completed
INFO:__main__:Data for BCS saved to C:\Users\phamt\Downloads\Thesis_Project\Dataset\BCS.csv


Batches:   0%|          | 0/40 [00:00<?, ?it/s]

ERROR:__main__:Error upserting data into Pinecone for BCS: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Sat, 28 Sep 2024 14:37:59 GMT', 'Content-Type': 'application/json', 'Content-Length': '113', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '707', 'x-pinecone-request-id': '4977348161307066030', 'x-envoy-upstream-service-time': '13', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"Number of provided vectors: 1258 exceeds the maximum amount per request: 1000","details":[]}

INFO:__main__:Processed data saved to C:\Users\phamt\Downloads\Thesis_Project\Processed_Dataset\processed_BCS.csv
[*********************100%***********************]  1 of 1 completed
INFO:__main__:Data for UAA saved to C:\Users\phamt\Downloads\Thesis_Project\Dataset\UAA.csv


Batches:   0%|          | 0/40 [00:00<?, ?it/s]

ERROR:__main__:Error upserting data into Pinecone for UAA: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Sat, 28 Sep 2024 14:38:17 GMT', 'Content-Type': 'application/json', 'Content-Length': '113', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '773', 'x-pinecone-request-id': '8646688259540232801', 'x-envoy-upstream-service-time': '14', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"Number of provided vectors: 1258 exceeds the maximum amount per request: 1000","details":[]}

INFO:__main__:Processed data saved to C:\Users\phamt\Downloads\Thesis_Project\Processed_Dataset\processed_UAA.csv


In [8]:
csv_files = ['NVDA.csv', 'INTC.csv', 'PLTR.csv', 'TSLA.csv', 'AAPL.csv', 'BBD.csv', 'T.csv', 'SOFI.csv',
             'WBD.csv', 'SNAP.csv', 'NIO.csv', 'BTG.csv', 'F.csv', 'AAL.csv', 'NOK.csv', 'BAC.csv',
             'CCL.csv', 'ORCL.csv', 'AMD.csv', 'PFE.csv', 'KGC.csv', 'MARA.csv', 'SLB.csv', 'NU.csv',
             'MPW.csv', 'MU.csv', 'LCID.csv', 'NCLH.csv', 'RIG.csv', 'AMZN.csv', 'ABEV.csv', 'U.csv',
             'LUMN.csv', 'AGNC.csv', 'VZ.csv', 'WBA.csv', 'WFC.csv', 'RIVN.csv', 'UPST.csv', 'GRAB.csv',
             'CSCO.csv', 'VALE.csv', 'AVGO.csv', 'PBR.csv', 'GOOGL.csv', 'SMMT.csv', 'GOLD.csv',
             'CMG.csv', 'BCS.csv', 'UAA.csv']

In [9]:
# Kiểm tra các tệp có trong thư mục
csv_files_in_folder = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Lấy các mã từ tên các file CSV
csv_symbols_in_folder = [f.split('.')[0] for f in csv_files_in_folder]

# Kiểm tra các mã cổ phiếu nào thiếu
missing_symbols = [symbol for symbol in symbols if symbol not in csv_symbols_in_folder]

# In ra kết quả
if missing_symbols:
    print(f"Các mã cổ phiếu bị thiếu: {missing_symbols}")
else:# Hybird Model - LSTM_SVR
    print("Đã có đủ dữ liệu cho tất cả 50 mã cổ phiếu.")

Đã có đủ dữ liệu cho tất cả 50 mã cổ phiếu.


# Hybird Model - LSTM_SVR

In [10]:
import os
import math
import numpy as np
import pandas as pd
from datetime import timedelta
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense

In [11]:
# Đọc dữ liệu từ CSV và chuẩn bị
def load_stock_data(symbol):
    file_path = os.path.join(folder_path, f'{symbol}.csv')  # Thêm '.csv' vào cuối tên tệp
    df = pd.read_csv(file_path, parse_dates=['Date'])
    df = df[['Date', 'Close']]  # Chỉ giữ lại cột Date và Close
    return df

In [12]:
def prepare_data(df, time_step=60):
    data = df['Close'].values.reshape(-1, 1)
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)

    x_data, y_data = [], []
    for i in range(time_step, len(scaled_data)):
        x_data.append(scaled_data[i-time_step:i, 0])
        y_data.append(scaled_data[i, 0])

    x_data, y_data = np.array(x_data), np.array(y_data)
    x_data = np.reshape(x_data, (x_data.shape[0], x_data.shape[1], 1))  # Thêm chiều cho LSTM
    return x_data, y_data, scaler

In [13]:
def build_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(input_shape[0], 1)))
    model.add(Dropout(0.2))
    model.add(LSTM(units=50, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(units=25))
    model.add(Dense(units=1))  # Output layer dự đoán giá đóng cửa

    # Compile mô hình
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [14]:
# Initialize list to store forecast results
forecast_results = []

# Iterate over each symbol
for symbol in symbols:
    print(f"Processing {symbol}...")
    df = load_stock_data(symbol)
    x_data, y_data, scaler = prepare_data(df)

    # Split data into train and test sets (80:20 ratio)
    train_size = int(len(x_data) * 0.8)
    x_train, x_test = x_data[:train_size], x_data[train_size:]
    y_train, y_test = y_data[:train_size], y_data[train_size:]

    # Rescale y_train for consistency
    y_train_rescaled = scaler.inverse_transform(y_train.reshape(-1, 1))

    # Build and train the LSTM model
    lstm_model = build_lstm_model(x_train.shape)
    lstm_model.fit(x_train, y_train, batch_size=64, epochs=10, verbose=1)

    # LSTM Predictions
    lstm_predictions = lstm_model.predict(x_test)
    lstm_predictions_rescaled = scaler.inverse_transform(lstm_predictions)

    # Rescale y_test to original values
    y_test_rescaled = scaler.inverse_transform(y_test.reshape(-1, 1))

    # Calculate residuals (actual - predicted)
    residuals = y_test_rescaled - lstm_predictions_rescaled

    # Flatten x_train and x_test for SVR
    x_train_flat = x_train[:, :, 0]
    x_test_flat = x_test[:, :, 0]

    # Ensure residuals and x_test_flat have the same length
    if len(x_test_flat) != len(residuals):
        residuals = residuals[:len(x_test_flat)]

    # Train SVR on residuals
    svr_model = SVR(kernel='rbf')
    svr_model.fit(x_train_flat, y_train)  # Train SVR with y_train

    # Predict residuals using SVR
    svr_residual_pred = svr_model.predict(x_test_flat).reshape(-1, 1)

    # Combine LSTM and SVR predictions
    final_predictions = lstm_predictions_rescaled + svr_residual_pred

    # Evaluate the model
    mse = mean_squared_error(y_test_rescaled, final_predictions)
    rmse = math.sqrt(mse)
    mape = mean_absolute_percentage_error(y_test_rescaled, final_predictions)

    print(f"RMSE for {symbol}: {rmse}")
    print(f"MSE for {symbol}: {mse}")
    print(f"MAPE for {symbol}: {mape}")

    # Forecast the next 365 days
    future_prices = []
    last_sequence = x_test[-1]  # Use the last sequence from the test set
    current_date = df['Date'].max()

    for i in range(365):  # Forecast 365 days
        lstm_next_pred = lstm_model.predict(last_sequence.reshape(1, -1, 1))
        svr_next_residual = svr_model.predict(last_sequence[:, 0].reshape(1, -1))
        next_price = lstm_next_pred + svr_next_residual

        # Inverse transform to get actual price
        next_price_rescaled = scaler.inverse_transform(next_price.reshape(-1, 1))
        future_prices.append(next_price_rescaled[0, 0])

        # Update the last_sequence with the new prediction
        last_sequence = np.append(last_sequence[1:], lstm_next_pred, axis=0)
        current_date += timedelta(days=1)

    # Compile forecast results for the current symbol
    forecast_results.append({
        'Symbol': symbol,
        'RMSE': rmse,
        'MSE': mse,
        'MAPE': mape,
        'Train_Prices': y_train_rescaled.flatten().tolist(),  # Add Train_prices
        'Predicted_Prices': final_predictions.flatten().tolist(),
        'Actual_Prices': y_test_rescaled.flatten().tolist(),
        'Future_Price_Predictions': future_prices  # Forecast for next year
    })

# Save all forecast results to a JSON file
output_file = os.path.join(output_folder, 'forecast_summary_lstm_svr.json')
with open(output_file, 'w') as f:
    json.dump(forecast_results, f, indent=4)

print(f"Forecast summary saved to {output_file}")

Processing NVDA...
Epoch 1/10


C:\Users\Pham Ty\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 29ms/step - loss: 0.0015
Epoch 2/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 1.3658e-04
Epoch 3/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 9.5055e-05
Epoch 4/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 7.0263e-05
Epoch 5/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 7.6411e-05
Epoch 6/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 7.1490e-05
Epoch 7/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 6.7293e-05
Epoch 8/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 6.5074e-05
Epoch 9/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 5.8848e-05
Epoch 10/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 6.4785e-05
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step
RMSE for NVDA: 6.200527256675485
MSE for NVDA: 38.446538260775625
MAPE for NVDA: 0.05906445165840347
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━

C:\Users\Pham Ty\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - loss: 0.1021
Epoch 2/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 0.0080
Epoch 3/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.0042
Epoch 4/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.0040
Epoch 5/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.0038
Epoch 6/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.0036
Epoch 7/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.0035
Epoch 8/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.0029
Epoch 9/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.0029
Epoch 10/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.0030
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step
RMSE for INTC: 2.017621220558951
MSE for INTC: 4.070795389649792
MAPE for INTC: 0.04822783994209765
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━

...

C:\Users\Pham Ty\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


31/31 ━━━━━━━━━━━━━━━━━━━━ 4s 32ms/step - loss: 0.0591
Epoch 2/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - loss: 0.0049
Epoch 3/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 0.0033
Epoch 4/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 0.0032
Epoch 5/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 0.0030
Epoch 6/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 0.0026
Epoch 7/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 0.0023
Epoch 8/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - loss: 0.0025
Epoch 9/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 0.0024
Epoch 10/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 0.0022
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step
RMSE for BCS: 0.48878807910815103
MSE for BCS: 0.23891378627823612
MAPE for BCS: 0.04527737540813862
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━

C:\Users\Pham Ty\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


31/31 ━━━━━━━━━━━━━━━━━━━━ 3s 28ms/step - loss: 0.0649
Epoch 2/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 0.0046
Epoch 3/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 0.0035
Epoch 4/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 0.0027
Epoch 5/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 0.0025
Epoch 6/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 0.0028
Epoch 7/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - loss: 0.0022
Epoch 8/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 0.0023
Epoch 9/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 0.0022
Epoch 10/10
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - loss: 0.0020
16/16 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step
RMSE for UAA: 0.7008507363976828
MSE for UAA: 0.4911917547091743
MAPE for UAA: 0.07734720247415927
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━

NameError: name 'json' is not defined

In [16]:
# Tạo DataFrame từ danh sách kết quả
summary_hybird = pd.DataFrame(forecast_results)

In [18]:
import os
import pandas as pd
import json

# Giả sử bạn đã có DataFrame summary_hybird
# summary_hybird = pd.DataFrame(forecast_results)

# Đường dẫn tới thư mục chứa file CSV đầu ra
output_folder = r'C:\Users\Pham Ty\Desktop\Thesis-Predict\Dataset_processed_2'

# Tên file CSV đầu ra
output_file = os.path.join(output_folder, 'forecast_summary_lstm_svr.csv')

# Các cột chứa dữ liệu dạng list cần chuyển đổi thành chuỗi JSON
list_columns = ['Train_Prices', 'Predicted_Prices', 'Actual_Prices', 'Future_Price_Predictions']

# Kiểm tra và chuyển đổi các cột list thành chuỗi JSON
for col in list_columns:
    if col in summary_hybird.columns:
        summary_hybird[col] = summary_hybird[col].apply(lambda x: json.dumps(x))
    else:
        print(f"Cột '{col}' không tồn tại trong DataFrame.")

# Lưu DataFrame vào file CSV
summary_hybird.to_csv(output_file, index=False)

print(f"Đã lưu kết quả vào file: {output_file}")

Đã lưu kết quả vào file: C:\Users\Pham Ty\Desktop\Thesis-Predict\Dataset_processed_2\forecast_summary_lstm_svr.csv


In [17]:
summary_hybird

,Symbol,RMSE,MSE,MAPE,Train_Prices,Predicted_Prices,Actual_Prices,Future_Price_Predictions
0,NVDA,6.200527,38.446538,0.059064,"[0.4907500147819519, 0.48925000429153437, 0.48...","[13.192129903614118, 13.127215698849954, 13.12...","[12.51200008392334, 13.166999816894531, 13.208...","[120.6523563359246, 120.49425382701692, 119.96..."
1,INTC,2.017621,4.070795,0.048228,"[36.22999954223633, 35.91999816894531, 35.5600...","[28.110063367424765, 27.881843524223928, 27.77...","[26.969999313354492, 27.70000076293945, 27.639...","[38.88841100750572, 39.22586069295641, 39.5698..."
2,PLTR,1.845420,3.405575,0.050947,"[27.750000000000004, 25.6299991607666, 24.6599...","[18.169883391008156, 18.108245089875723, 18.06...","[17.840000152587887, 17.950000762939453, 17.25...","[54.38294129198328, 54.04788921989151, 53.7804..."
3,TSLA,19.774848,391.044609,0.081765,"[13.800000190734865, 13.602666854858398, 13.18...","[293.38954313179005, 288.7537847822936, 283.82...","[242.3999938964844, 249.44000244140625, 240.80...","[467.24798981569245, 470.39181608843114, 474.2..."
4,AAPL,7.055187,49.775662,0.031062,"[27.4325008392334, 27.0575008392334, 26.6875, ...","[150.5402848191565, 149.35343781971036, 148.37...","[142.44999694824216, 146.1000061035156, 146.39...","[306.56119949434407, 305.7557747723764, 304.80..."
5,BBD,0.291272,0.084839,0.092300,"[8.836088180541994, 8.443526268005371, 8.26446...","[3.9316881112257396, 3.944964084442645, 3.9753...","[4.059999942779541, 4.090000152587891, 4.11999...","[5.258950176924643, 5.251140310926963, 5.24575..."
6,T,0.745363,0.555566,0.035687,"[24.29002952575684, 24.35800552368164, 24.2749...","[16.63017049872987, 16.546171487922024, 16.489...","[15.899999618530272, 16.09000015258789, 15.930...","[27.716508450100676, 27.86204440557155, 27.995..."
7,SOFI,0.552099,0.304813,0.058962,"[17.14999961853027, 17.270000457763672, 16.719...","[9.837625969056164, 9.667475064992862, 9.48559...","[8.529999732971191, 8.319999694824219, 8.47999...","[10.856825703976659, 10.861114921429177, 10.87..."
8,WBD,1.634677,2.672171,0.150011,"[33.380001068115234, 32.869998931884766, 33.18...","[13.261179841407376, 13.175086248459767, 13.14...","[11.949999809265137, 12.359999656677246, 12.22...","[33.44752411172808, 33.53399662954772, 33.6527..."
9,SNAP,1.311206,1.719262,0.082298,"[21.21999931335449, 21.450000762939453, 21.209...","[11.647327618064372, 11.660599145363372, 11.67...","[11.210000038146973, 11.239999771118164, 11.00...","[17.894139598694952, 18.00361059456263, 18.119..."


In [2]:
import os
import pandas as pd
import numpy as np
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import logging
from datetime import timedelta
import json
from dash import dash_table

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s:%(message)s')

# Paths to data
folder_path = r'C:\Users\Pham Ty\Desktop\Thesis-Predict\Dataset-2'  # Folder with stock CSV files
output_folder = r'C:\Users\Pham Ty\Desktop\Thesis-Predict\Dataset_processed_2'  # Output folder

# List of stock symbols
symbols = ['NVDA', 'INTC', 'PLTR', 'TSLA', 'AAPL', 'BBD', 'T', 'SOFI',
           'WBD', 'SNAP', 'NIO', 'BTG', 'F', 'AAL', 'NOK', 'BAC',
           'CCL', 'ORCL', 'AMD', 'PFE', 'KGC', 'MARA', 'SLB', 'NU',
           'MPW', 'MU', 'LCID', 'NCLH', 'RIG', 'AMZN', 'ABEV', 'U',
           'LUMN', 'AGNC', 'VZ', 'WBA', 'WFC', 'RIVN', 'UPST', 'GRAB',
           'CSCO', 'VALE', 'AVGO', 'PBR', 'GOOGL', 'SMMT', 'GOLD',
           'CMG', 'BCS', 'UAA']

# Load forecast_summary_lstm_svr.csv
forecast_summary_file = os.path.join(output_folder, 'forecast_summary_lstm_svr.csv')
if os.path.exists(forecast_summary_file):
    forecast_summary_df = pd.read_csv(forecast_summary_file)
    # Parse string lists into actual lists using json.loads
    list_columns = ['Predicted_Prices', 'Actual_Prices', 'Future_Price_Predictions', 'Train_Prices']
    for col in list_columns:
        if col in forecast_summary_df.columns:
            forecast_summary_df[col] = forecast_summary_df[col].apply(
                lambda x: json.loads(x) if pd.notnull(x) else []
            )
    logging.info(f"Loaded '{forecast_summary_file}' successfully.")
else:
    forecast_summary_df = pd.DataFrame()
    logging.warning(f"Could not find '{forecast_summary_file}'. Evaluation metrics and forecasts will not be displayed.")

# Initialize Dash app
app = dash.Dash(__name__)
server = app.server  # For deployment

# Define app layout
app.layout = html.Div([
    html.H1('Stock Price Predictions Dashboard'),
    html.Div([
        html.Label('Select Stock Symbol'),
        dcc.Dropdown(
            id='stock-dropdown',
            options=[{'label': s, 'value': s} for s in symbols],
            value='NVDA'  # Default stock
        )
    ], style={'width': '25%', 'display': 'inline-block'}),
    html.Div([
        html.Label('Select Date Range'),
        dcc.DatePickerRange(
            id='date-picker',
            min_date_allowed='2014-09-18',  # Adjusted min date for 10-year period
            max_date_allowed='2025-09-18',  # Mở rộng thêm 1 năm cho dự báo
            start_date='2014-09-18',        # Default start date
            end_date='2025-09-18'           # Mở rộng thêm 1 năm cho dự báo
        )
    ], style={'display': 'inline-block', 'marginLeft': '50px'}),
    html.Div([
        html.Label('Show 1-Year Forecast'),
        dcc.Checklist(
            id='forecast-checkbox',
            options=[{'label': 'Include 1-Year Forecast', 'value': 'show_forecast'}],
            value=[],
            inline=True
        )
    ], style={'marginTop': '20px'}),
    dcc.Graph(id='price-graph'),
    html.Div(id='metrics-output', style={'marginTop': '20px'}),
    html.Div([
        html.Button("Download Predictions", id="download-button"),
        dcc.Download(id="download-predictions")
    ], style={'marginTop': '20px'}),
    html.Div(id='download-info', style={'marginTop': '10px'})
])

# Define callback to update graph and evaluation metrics
@app.callback(
    [Output('price-graph', 'figure'),
     Output('metrics-output', 'children')],
    [Input('stock-dropdown', 'value'),
     Input('date-picker', 'start_date'),
     Input('date-picker', 'end_date'),
     Input('forecast-checkbox', 'value')]
)
def update_graph(selected_stock, start_date, end_date, forecast_option):
    # Convert start_date and end_date to datetime
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)

    # Get the forecast data for the selected stock
    forecast_row = forecast_summary_df[forecast_summary_df['Symbol'] == selected_stock]

    if forecast_row.empty:
        logging.error(f"No forecast data found for {selected_stock}.")
        return {
            'data': [],
            'layout': go.Layout(
                title=f'{selected_stock} Price Prediction',
                xaxis={'title': 'Date'},
                yaxis={'title': 'Price'}
            )
        }, f"No forecast data found for {selected_stock}."

    # Parse 'Actual_Prices', 'Predicted_Prices', 'Future_Price_Predictions', 'Train_Prices'
    actual_prices = forecast_row.iloc[0]['Actual_Prices']
    predicted_prices = forecast_row.iloc[0]['Predicted_Prices']
    future_prices = forecast_row.iloc[0]['Future_Price_Predictions']
    train_prices = forecast_row.iloc[0]['Train_Prices']

    # Kiểm tra dữ liệu future_prices
    if not future_prices:
        logging.warning(f"No future price predictions available for {selected_stock}.")
    else:
        logging.info(f"Loaded {len(future_prices)} future price predictions for {selected_stock}.")

    # Load the stock data to get dates
    stock_data_file = os.path.join(folder_path, f'{selected_stock}.csv')
    if not os.path.exists(stock_data_file):
        logging.error(f"Stock data file not found for {selected_stock}.")
        return {
            'data': [],
            'layout': go.Layout(
                title=f'{selected_stock} Price Prediction',
                xaxis={'title': 'Date'},
                yaxis={'title': 'Price'}
            )
        }, f"Stock data file not found for {selected_stock}."

    df_stock = pd.read_csv(stock_data_file)
    if 'Date' not in df_stock.columns or 'Close' not in df_stock.columns:
        logging.error(f"Incorrect data format in stock data file for {selected_stock}.")
        return {
            'data': [],
            'layout': go.Layout(
                title=f'{selected_stock} Price Prediction',
                xaxis={'title': 'Date'},
                yaxis={'title': 'Price'}
            )
        }, f"Incorrect data format in stock data file for {selected_stock}."

    # Convert 'Date' to datetime
    df_stock['Date'] = pd.to_datetime(df_stock['Date'])
    dates = df_stock['Date']
    data_length = len(df_stock)

    time_step = 60  # As used in your model
    total_samples = data_length - time_step

    # Calculate train_size and test_size based on 10-year data
    # Assuming 8 years for training and 2 years for testing
    # Calculate the number of samples per year
    # For simplicity, assume 252 trading days per year
    samples_per_year = 252
    train_size = 8 * samples_per_year
    test_size = 2 * samples_per_year

    # Adjust train_size and test_size if total_samples is less
    if train_size + test_size > total_samples:
        train_size = int(total_samples * 0.8)
        test_size = total_samples - train_size
        logging.warning(f"Adjusted train_size to {train_size} and test_size to {test_size} due to insufficient data.")

    # Ensure that the length of 'Actual_Prices' matches 'test_size'
    if len(actual_prices) != test_size or len(predicted_prices) != test_size:
        logging.error(f"Length mismatch between actual/predicted prices and test set for {selected_stock}.")
        return {
            'data': [],
            'layout': go.Layout(
                title=f'{selected_stock} Price Prediction',
                xaxis={'title': 'Date'},
                yaxis={'title': 'Price'}
            )
        }, f"Length mismatch in data for {selected_stock}."

    # Get indices for train and test sets in original data
    train_indices_in_y_data = range(0, train_size)
    test_indices_in_y_data = range(train_size, train_size + test_size)
    indices_train = [i + time_step for i in train_indices_in_y_data]
    indices_test = [i + time_step for i in test_indices_in_y_data]

    # Ensure indices do not exceed the length of dates
    indices_train = [i for i in indices_train if i < len(dates)]
    indices_test = [i for i in indices_test if i < len(dates)]

    # Dates for train and test sets
    dates_train = dates.iloc[indices_train].reset_index(drop=True)
    dates_test = dates.iloc[indices_test].reset_index(drop=True)

    # Create DataFrame for train set
    df_train = pd.DataFrame({
        'Date': dates_train,
        'Train_Price': train_prices
    })

    # Create DataFrame for test set
    df_test = pd.DataFrame({
        'Date': dates_test,
        'Actual_Price': actual_prices,
        'Predicted_Price': predicted_prices
    })

    # Filter df_train and df_test based on 'start_date' and 'end_date'
    df_train_filtered = df_train[(df_train['Date'] >= start_date) & (df_train['Date'] <= end_date)]
    df_test_filtered = df_test[(df_test['Date'] >= start_date) & (df_test['Date'] <= end_date)]

    # Initialize data list for plotting
    data = []

    # Add Train Prices if available in the selected date range
    if not df_train_filtered.empty:
        data.append(
            go.Scatter(
                x=df_train_filtered['Date'],
                y=df_train_filtered['Train_Price'],
                mode='lines',
                name='Train Price',
                line=dict(color='red')
            )
        )

    # Add Actual Test Prices
    if not df_test_filtered.empty:
        data.append(
            go.Scatter(
                x=df_test_filtered['Date'],
                y=df_test_filtered['Actual_Price'],
                mode='lines',
                name='Actual Test Price',
                line=dict(color='green')
            )
        )

    # Add Predicted Test Prices
    if not df_test_filtered.empty:
        data.append(
            go.Scatter(
                x=df_test_filtered['Date'],
                y=df_test_filtered['Predicted_Price'],
                mode='lines',
                name='Predicted Test Price',
                line=dict(color='blue')
            )
        )

    # Add future price predictions if selected
    if 'show_forecast' in forecast_option and future_prices:
        # Generate future dates
        last_date = dates.iloc[-1]
        future_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=len(future_prices), freq='D')

        if len(future_prices) != len(future_dates):
            logging.error(f"Length mismatch between future prices and future dates for {selected_stock}.")
        else:
            df_future = pd.DataFrame({
                'Date': future_dates,
                'Future_Price': future_prices
            })
            # Với việc mở rộng DatePickerRange, không cần lọc ngày nữa
            df_future_filtered = df_future[(df_future['Date'] >= start_date) & (df_future['Date'] <= end_date)]
            if not df_future_filtered.empty:
                data.append(
                    go.Scatter(
                        x=df_future_filtered['Date'],
                        y=df_future_filtered['Future_Price'],
                        mode='lines',
                        name='1-Year Forecast',
                        line=dict(dash='dash', color='orange')
                    )
                )

    # Create figure
    figure = {
        'data': data,
        'layout': go.Layout(
            title=f'{selected_stock} Price Prediction',
            xaxis={'title': 'Date'},
            yaxis={'title': 'Price'},
            hovermode='closest'
        )
    }

    # Get RMSE, MSE, and MAPE
    rmse = forecast_row.iloc[0]['RMSE']
    mse = forecast_row.iloc[0]['MSE']
    mape = forecast_row.iloc[0]['MAPE']

    # Create metrics DataFrame
    metrics_df = pd.DataFrame({
        'Metric': ['RMSE', 'MSE', 'MAPE'],
        'Value': [f"{rmse:.4f}", f"{mse:.4f}", f"{mape:.2%}"]
    })

    metrics_output = html.Div([
        html.H4('Evaluation Metrics'),
        dash_table.DataTable(
            columns=[{"name": i, "id": i} for i in metrics_df.columns],
            data=metrics_df.to_dict('records'),
            style_cell={'textAlign': 'left'},
            style_header={
                'backgroundColor': 'rgb(230, 230, 230)',
                'fontWeight': 'bold'
            },
            style_table={'width': '50%'}
        )
    ])

    return figure, metrics_output

# Define callback for downloading predictions
@app.callback(
    Output("download-predictions", "data"),
    [Input("download-button", "n_clicks")],
    [dash.dependencies.State('stock-dropdown', 'value')],
    prevent_initial_call=True,
)
def download_predictions(n_clicks, selected_stock):
    if n_clicks is None:
        return dash.no_update

    # Get the forecast data for the selected stock
    forecast_row = forecast_summary_df[forecast_summary_df['Symbol'] == selected_stock]

    if forecast_row.empty:
        return dash.no_update

    # Extract data
    actual_prices = forecast_row.iloc[0]['Actual_Prices']
    predicted_prices = forecast_row.iloc[0]['Predicted_Prices']
    future_prices = forecast_row.iloc[0]['Future_Price_Predictions']
    train_prices = forecast_row.iloc[0]['Train_Prices']

    # Load stock dates
    stock_data_file = os.path.join(folder_path, f'{selected_stock}.csv')
    df_stock = pd.read_csv(stock_data_file)
    df_stock['Date'] = pd.to_datetime(df_stock['Date'])
    dates = df_stock['Date']

    # Determine train and test dates
    time_step = 60
    total_samples = len(dates) - time_step
    samples_per_year = 252
    train_size = 8 * samples_per_year
    test_size = 2 * samples_per_year
    if train_size + test_size > total_samples:
        train_size = int(total_samples * 0.8)
        test_size = total_samples - train_size

    indices_test = range(train_size, train_size + test_size)
    indices_test = [i + time_step for i in indices_test if (i + time_step) < len(dates)]
    dates_test = dates.iloc[indices_test].reset_index(drop=True)

    # Prepare data for download
    download_df = pd.DataFrame({
        'Date': dates_test,
        'Actual_Price': actual_prices,
        'Predicted_Price': predicted_prices
    })

    # Add future predictions if any
    if future_prices:
        last_date = dates.iloc[-1]
        future_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=len(future_prices), freq='D')
        download_future_df = pd.DataFrame({
            'Date': future_dates,
            'Future_Price_Prediction': future_prices
        })
        download_df = pd.concat([download_df, download_future_df], ignore_index=True)

    # Convert DataFrame to CSV
    return dcc.send_data_frame(download_df.to_csv, f'{selected_stock}_predictions.csv', index=False)

# Run Dash app
if __name__ == '__main__':
    app.run_server(debug=False, use_reloader=False)

2024-10-21 21:12:43,424 INFO:Loaded 'C:\Users\Pham Ty\Desktop\Thesis-Predict\Dataset_processed_2\forecast_summary_lstm_svr.csv' successfully.


2024-10-21 21:12:45,221 INFO:Loaded 365 future price predictions for NVDA.
2024-10-21 21:12:45,234 WARNING:Adjusted train_size to 1964 and test_size to 492 due to insufficient data.
2024-10-21 21:12:48,667 INFO:Loaded 365 future price predictions for NVDA.
2024-10-21 21:12:48,676 WARNING:Adjusted train_size to 1964 and test_size to 492 due to insufficient data.
2024-10-21 21:12:51,786 INFO:Loaded 365 future price predictions for NVDA.
2024-10-21 21:12:51,794 WARNING:Adjusted train_size to 1964 and test_size to 492 due to insufficient data.
2024-10-21 21:12:53,326 INFO:Loaded 365 future price predictions for NVDA.
2024-10-21 21:12:53,335 WARNING:Adjusted train_size to 1964 and test_size to 492 due to insufficient data.
2024-10-21 21:12:53,994 INFO:Loaded 365 future price predictions for NVDA.
2024-10-21 21:12:54,003 WARNING:Adjusted train_size to 1964 and test_size to 492 due to insufficient data.
2024-10-21 21:15:19,838 INFO:Loaded 365 future price predictions for NVDA.
2024-10-21 21:1